In [2]:
# Import necessary libraries
import sqlite3
import pandas as pd

# Connect to a SQLite database (or create one)
conn = sqlite3.connect(r'C:\Users\spiri\codingbootcamp\Final-project-C4Y\data.db')
cursor = conn.cursor()

In [4]:
cursor.execute("SELECT * FROM balances limit 5")
rows = cursor.fetchall()

for row in rows:
    print(row)

('2023-06-06 16:21:15.291346', 2680.467772955898, 249789938, 'CZK')
('2023-05-23 16:21:15.291346', 2654.0554191231317, 249789938, 'CZK')
('2023-05-09 16:21:15.291346', 2620.8358717314222, 249789938, 'CZK')
('2023-04-25 16:21:15.291346', 2720.3159250829763, 249789938, 'CZK')
('2023-04-11 16:21:15.291346', 2687.6019824322448, 249789938, 'CZK')


In [9]:
cursor.execute("SELECT count(distinct client_id)  FROM balances   ")
rows = cursor.fetchall()

for row in rows:
    print(row)

(11162,)


In [35]:
cursor.execute('''
SELECT COUNT(DISTINCT currency) AS num_currencies, client_id 
FROM balances 
GROUP BY client_id 
HAVING num_currencies = 2;''')
rows = cursor.fetchall()

for row in rows:
    print(row)

In [36]:
cursor.execute('''
WITH mean_deposite as (
    select client_id, AVG(balance), currency
    from balances
    group by 1
)
Select * 
from inv_campaign_eval
left join client
on inv_campaign_eval.client_id = client.client_id
left join client_products
on inv_campaign_eval.client_id = client_products.client_id
left join mean_deposite
on inv_campaign_eval.client_id = mean_deposite.client_id''')
rows = cursor.fetchall()

for row in rows:
    print(row)

(249789938, 'success', 249789938, 38, 'services', 'married', 'secondary', 'M', 249789938, 'yes', 'no', 'n', 'no', 249789938, 2669.532734122098, 'CZK')
(1504633819, 'failure', 1504633819, 38, 'management', 'married', 'tertiary', 'M', 1504633819, 'no', 'yes', 'n', 'yes', 1504633819, 5440.862334416502, 'CZK')
(1952195738, 'success', 1952195738, 34, 'services', 'married', 'secondary', 'M', 1952195738, 'yes', 'yes', 'n', 'no', 1952195738, 2922.9588516124836, 'CZK')
(989726384, 'success', 989726384, 62, 'retired', 'married', 'secondary', 'F', 989726384, 'yes', 'no', 'n', 'no', 989726384, 3124.6148425378415, 'CZK')
(1971878545, 'failure', 1971878545, 36, 'management', 'single', 'tertiary', 'M', 1971878545, 'yes', 'no', 'n', 'no', 1971878545, 474.08303261699086, 'CZK')
(1823731216, 'failure', 1823731216, 56, 'admin.', 'single', 'secondary', 'F', 1823731216, 'no', 'no', 'y', 'yes', 1823731216, 1848.2416752867825, 'CZK')
(1129179996, 'failure', 1129179996, 39, 'services', 'married', 'secondary',